# Axial actuators No back driving analysis

This notebook analyzes the NO back driving data of the M2 axial actuators.

The test has been performed with the configuration file: **ts_mtm2/config/sysconfig
/Configurable_File_Description_20180831T092556_surrogate_handling.csv**.

The goal is to verify that:

1. The axial actuators do no back drive when the M2 cell is powered off
2. The M2 mirror does not drift in position when the power is cut.

## Import Modules

This notebook needs to setup the **ts_m2com** and **ts_aos_utilsts** under the **notebooks/.user_setups**, which depends on the **ts_tcpip**.
You also need to have **ts_mtm2** under the **WORK/** directory to read the confiugration files to do the analysis.

In [ ]:
%matplotlib inline
%matplotlib widget
import matplotlib.pyplot as plt
import math
from pathlib import Path
import numpy as np
from sklearn.linear_model import LinearRegression
from astropy.time import Time
import pandas as pd
import lsst_efd_client

from lsst.ts.m2com import MockControlClosedLoop
import tabulate 
from lsst.ts.aos.utils import DiagnosticsM2, EfdName

In [ ]:
# function to retrieve EFD topics

async def query_data(
    diagnostics_m2: DiagnosticsM2,
    control_closed_loop: MockControlClosedLoop,
    time_start: Time,
    time_end: Time,
) -> None:
    """
    Query the data.

    # functions declaration can be found here:
    # https://github.com/lsst-ts/ts_aos_utils/blob/feature/m2power/python/lsst/ts/aos/utils/diagnostics_m2.py
    Parameters
    ----------
    diagnostics_m2 : `lsst.ts.aos.utils.DiagnosticsM2`
        M2 diagnostics instance.
    control_closed_loop : `lsst.ts.m2.com.MockControlClosedLoop`
        Mock control closed loop instance.
    time_start : `astropy.time.core.Time`
        Start time.
    time_end : `astropy.time.core.Time`
        End time.
    """

    # Get the x, y position of actuators
    xy_actuators = diagnostics_m2.get_xy_actuators(control_closed_loop)

    # Query data
    data_ims, time_operation_ims = await diagnostics_m2.get_data_position(
        "positionIMS", time_start, time_end
    )

    data_collected_axial, data_collected_tangent = await diagnostics_m2.get_data_force(
        time_start, time_end
    )

    data_power_status, time_operation = await diagnostics_m2.get_data_power_status(
        time_start, time_end
    )

    return xy_actuators, data_ims, data_collected_axial, data_power_status

In [ ]:
# function that prints variable values at terminal for sanity check

def print_data(
    data_collected_axial, data_ims, data_power_status, actuators_groups, group, face
):
    # number elements force vector
    # convert tuple into integer
    temp = np.shape(data_collected_axial["measured"][:, 0])
    y = "".join(map(str, temp))
    n_elem_force = int(y)

    # time interval data stream
    delta_t = data_ims.index[-1] - data_ims.index[0]

    print(data_ims.index[0])
    print(data_ims.index[-1])
    print(f"Signal length, sec: {delta_t.seconds}")

    fig = plt.figure()
    ax = fig.add_subplot(111)
    ax.plot(data_power_status.index, data_power_status.motorCurrent)
    ax.set_xlabel("UTC")
    ax.set_ylabel("Motor Current, A")


    
    fig.tight_layout()

    # array number of elements
    n_elem_motorcurrent = len(data_power_status.motorCurrent)
    n_elem_ims = len(data_ims.x)

    # scaling factors
    relative_scales = n_elem_force / n_elem_motorcurrent
    relative_scales_ims = n_elem_ims / n_elem_motorcurrent
    relative_scales_ims_force = n_elem_force / n_elem_ims

    # find time of force command
    for idx in range(len(data_collected_axial["measured"][:, actuators_groups[0]])):
        if idx == len(data_collected_axial["measured"][:, actuators_groups[0]]) - 1:
            break

        delta = abs(
            data_collected_axial["measured"][idx + 1, actuators_groups[0]]
            - data_collected_axial["measured"][idx, actuators_groups[0]]
        )

        if (face == "down" and abs(delta) > 100) or (face == "up" and abs(delta) > 70):
            break

        force_command_time = idx

    print(f"force command time: {force_command_time}")

    # some integers are added to the times to make sure the command upload is completed
    force_command_time = force_command_time + 4

    command_time_ims = round(force_command_time / relative_scales_ims_force) + 50

    # find time of Estop push
    estop_push = np.where(data_power_status.motorCurrent == 0)[0]

    time_estop = data_power_status.index[estop_push[0]]

    eStop = round(estop_push[0] * relative_scales)
    eStop_ims = round(estop_push[0] * relative_scales_ims) + 10
    print(f"E-stop push time: {eStop}")

    # number of data points in the interval
    full_stream = len(data_collected_axial["measured"])
    print(f"Force data points: {full_stream}")

    # **************************************
    #
    #  MEASURED FORCES after Estop push
    #
    # **************************************

    statistics_pre_estop = np.zeros((6, 12))
    statistics_post_estop = np.zeros((6, 12))

    n = 12
    for idx in range(n):
        print(f"actuator: {actuators_groups[idx]+1}")

        # remove the average value from each signal
        avg_signal_pre = (
            data_collected_axial["measured"][
                force_command_time:eStop, actuators_groups[idx]
            ]
            - data_collected_axial["measured"][
                force_command_time:eStop, actuators_groups[idx]
            ].mean()
        )
        avg_signal_post = (
            data_collected_axial["measured"][eStop:full_stream, actuators_groups[idx]]
            - data_collected_axial["measured"][
                eStop:full_stream, actuators_groups[idx]
            ].mean()
        )

        # PRE
        statistics_pre_estop[0, idx] = avg_signal_pre.max()
        statistics_pre_estop[1, idx] = avg_signal_pre.min()
        statistics_pre_estop[2, idx] = avg_signal_pre.max() - avg_signal_pre.min()
        statistics_pre_estop[4, idx] = math.sqrt(
            sum((avg_signal_pre) ** 2 / len(avg_signal_pre))
        )

        # POST
        statistics_post_estop[0, idx] = avg_signal_post.max()
        statistics_post_estop[1, idx] = avg_signal_post.min()
        statistics_post_estop[2, idx] = avg_signal_post.max() - avg_signal_post.min()
        statistics_post_estop[4, idx] = math.sqrt(
            sum((avg_signal_post) ** 2 / len(avg_signal_post))
        )

    return (
        full_stream,
        statistics_pre_estop,
        statistics_post_estop,
        eStop,
        eStop_ims,
        time_estop,
        force_command_time,
        command_time_ims,
    )

In [ ]:
# function that plots actuator forces

def actuator_plot(
    diagnostics_m2,
    control_closed_loop,
    force_command_time,
    actuators_groups,
    group,
    eStop,
    data_collected_axial,
    full_stream,
    statistics_pre_estop,
    statistics_post_estop,
    face,
):
    # Plot map actutors activated

    xy_actuators = diagnostics_m2.get_xy_actuators(control_closed_loop)
    max_force_axial = np.max(
        np.abs(data_collected_axial["measured"][force_command_time + 10, :])
    )
    force_axial = data_collected_axial["measured"][force_command_time + 10, :]
    max_marker_size: int = 300
    NUM_TANGENT_LINK = 6

    magnification_axial = (
        max_marker_size / max_force_axial if max_force_axial != 0 else 1
    )

    fig = plt.figure()
    fig.suptitle(f"ACTUATORS GROUP {group+1} \nAxial actuators maps")
    ax = plt.subplot(111)

    # Draw the axial actuators
    img = ax.scatter(
        xy_actuators[:-NUM_TANGENT_LINK, 0],
        xy_actuators[:-NUM_TANGENT_LINK, 1],
        s=np.abs(force_axial) * magnification_axial,
        c=force_axial,
        vmin=min(force_axial),
        vmax=max(force_axial),
    )
    fig.colorbar(img, ax=ax)

    ax.set_xlabel("X position (m)")
    ax.set_ylabel("Y position (m)")

    fig.tight_layout()

    # graphics of forces before and after Estop push
    font = {
        "family": "serif",
        "color": "black",
        "weight": "normal",
        "size": 10,
    }

    hatF = "$\overline{F}), N$"

    # create new label for X axis

    fig = plt.figure()
    fig.suptitle(f"ACTUATORS GROUP {group+1}")
    fig.subplots_adjust(hspace=0.9, wspace=0.9)

    # line of Estop
    x_eStop = np.ones((5,), dtype=int)
    x_eStop = x_eStop * eStop

    if face == "down":
        y_eStop = np.array([-20, 50, 100, 200, 420])
    else:
        y_eStop = np.array([-400, -150, -100, -50, 50])

    # line force command
    x_force = np.ones((5,), dtype=int)
    x_force = x_force * force_command_time

    if face == "down":
        y_force = np.array([-20, 50, 100, 200, 420])
    else:
        y_force = np.array([-400, -150, -100, -50, 50])

    n = 12
    for idx in range(n):
        ax = fig.add_subplot(4, 3, idx + 1)
        ax.plot(data_collected_axial["measured"][:, actuators_groups[idx]])
        ax.plot(x_force, y_force)
        ax.plot(x_eStop, y_eStop)
        num_act = actuators_groups[idx] + 1
        ax.set_title("B=%i" % num_act, fontdict=font)
        ax.set_ylabel("Meas. F, N", fontdict=font)

        if face == "down":
            ax.set_yticks([0, 250, 400], ["0", "250", "400"])
        else:
            ax.set_yticks([-400, -200, 50], ["-400", "-200", "50"])

    ax1 = plt.subplot(431)
    ax1.plot(x_force, y_force, color="orange", label="Force command")
    ax1.plot(x_eStop, y_eStop, color="green", label="Estop")
    ax1.legend(loc="lower right", fontsize="5")

    # average forces before and after E-stop push
    fig = plt.figure()
    fig.suptitle(f"ACTUATORS GROUP {group+1}")
    fig.subplots_adjust(hspace=0.9, wspace=0.9)
    hatF = "Meas. $\overline{F}, N$"

    n = 12
    for idx in range(n):
        samples_pre = data_collected_axial["measured"][
            force_command_time:eStop, actuators_groups[idx]
        ]
        x_pre = np.linspace(0, len(samples_pre), num=len(samples_pre))

        samples_post = data_collected_axial["measured"][
            eStop:full_stream, actuators_groups[idx]
        ]
        x_post = np.linspace(0, len(samples_post), num=len(samples_post))

        rms_pre = np.ones(len(x_pre)) * statistics_pre_estop[4, idx]
        rms_post = np.ones(len(x_post)) * statistics_post_estop[4, idx]

        ax = fig.add_subplot(4, 3, idx + 1)
        ax.errorbar(x_pre, samples_pre, rms_pre, fmt="blue", ecolor="gray", lw=0.5)
        ax.errorbar(x_post, samples_post, rms_post, fmt="orange", ecolor="gray", lw=0.5)
        num_act = actuators_groups[idx] + 1
        ax.set_title("B=%i" % num_act, fontdict=font)
        ax.set_ylabel(hatF, fontdict=font)

    fig.tight_layout()

In [ ]:
# Rigid body position from IMS after E-stop

def ims_estop(
    data_ims,
    command_time_ims,
    eStop_ims,
    group,
):
    # graphics of forces before and after Estop push
    font = {
        "family": "serif",
        "color": "black",
        "weight": "normal",
        "size": 10,
    }
    # average IMS positions before and after E-stop push

    # pre Estop, post force command

    # Mean
    samples_pre_x = pd.DataFrame(data_ims.x[command_time_ims:eStop_ims]).values.mean()
    samples_pre_y = pd.DataFrame(data_ims.y[command_time_ims:eStop_ims]).values.mean()
    samples_pre_z = pd.DataFrame(data_ims.z[command_time_ims:eStop_ims]).values.mean()
    samples_pre_xrot = pd.DataFrame(
        data_ims.xRot[command_time_ims:eStop_ims]
    ).values.mean()
    samples_pre_yrot = pd.DataFrame(
        data_ims.yRot[command_time_ims:eStop_ims]
    ).values.mean()
    samples_pre_zrot = pd.DataFrame(
        data_ims.zRot[command_time_ims:eStop_ims]
    ).values.mean()

    # RMS
    rms_x_pre_estop = pd.DataFrame(data_ims.x[command_time_ims:eStop_ims]).values.std()
    rms_y_pre_estop = pd.DataFrame(data_ims.y[command_time_ims:eStop_ims]).values.std()
    rms_z_pre_estop = pd.DataFrame(data_ims.z[command_time_ims:eStop_ims]).values.std()
    rms_xrot_pre_estop = pd.DataFrame(
        data_ims.xRot[command_time_ims:eStop_ims]
    ).values.std()
    rms_yrot_pre_estop = pd.DataFrame(
        data_ims.yRot[command_time_ims:eStop_ims]
    ).values.std()
    rms_zrot_pre_estop = pd.DataFrame(
        data_ims.zRot[command_time_ims:eStop_ims]
    ).values.std()

    # post Estop

    # Mean
    samples_post_x = pd.DataFrame(data_ims.x[eStop_ims : len(data_ims.x)]).values.mean()
    samples_post_y = pd.DataFrame(data_ims.y[eStop_ims : len(data_ims.x)]).values.mean()
    samples_post_z = pd.DataFrame(data_ims.z[eStop_ims : len(data_ims.x)]).values.mean()
    samples_post_xrot = pd.DataFrame(
        data_ims.xRot[eStop_ims : len(data_ims.x)]
    ).values.mean()
    samples_post_yrot = pd.DataFrame(
        data_ims.yRot[eStop_ims : len(data_ims.x)]
    ).values.mean()
    samples_post_zrot = pd.DataFrame(
        data_ims.zRot[eStop_ims : len(data_ims.x)]
    ).values.mean()

    # RMS
    rms_x_post_estop = pd.DataFrame(
        data_ims.x[eStop_ims : len(data_ims.x)]
    ).values.std()
    rms_y_post_estop = pd.DataFrame(
        data_ims.y[eStop_ims : len(data_ims.x)]
    ).values.std()
    rms_z_post_estop = pd.DataFrame(
        data_ims.z[eStop_ims : len(data_ims.x)]
    ).values.std()
    rms_xrot_post_estop = pd.DataFrame(
        data_ims.xRot[eStop_ims : len(data_ims.x)]
    ).values.std()
    rms_yrot_post_estop = pd.DataFrame(
        data_ims.yRot[eStop_ims : len(data_ims.x)]
    ).values.std()
    rms_zrot_post_estop = pd.DataFrame(
        data_ims.zRot[eStop_ims : len(data_ims.x)]
    ).values.std()

    ims_x = np.array([samples_pre_x, samples_post_x])
    ims_y = np.array([samples_pre_y, samples_post_y])
    ims_z = np.array([samples_pre_z, samples_post_z])
    ims_xrot = np.array([samples_pre_xrot, samples_post_xrot])
    ims_yrot = np.array([samples_pre_yrot, samples_post_yrot])
    ims_zrot = np.array([samples_pre_zrot, samples_post_zrot])

    rms_x = np.array([rms_x_pre_estop, rms_x_post_estop])
    rms_y = np.array([rms_y_pre_estop, rms_y_post_estop])
    rms_z = np.array([rms_z_pre_estop, rms_z_post_estop])
    rms_xrot = np.array([rms_xrot_pre_estop, rms_xrot_post_estop])
    rms_yrot = np.array([rms_yrot_pre_estop, rms_yrot_post_estop])
    rms_zrot = np.array([rms_zrot_pre_estop, rms_zrot_post_estop])

    x = [data_ims.index[command_time_ims], data_ims.index[eStop_ims]]

    req_linear = 0.5
    req_angular = 3e-5 * 3600 / 2

    xstr = np.array([f"{el.hour}:{el.minute}:{el.second}" for el in x])
    xstr1 = xstr[0] + " E-stop"
    xstr2 = xstr[1] + " End test"
    xstring = [xstr1, xstr2]

    fig = plt.figure()
    fig.suptitle(f"ACTUATORS GROUP {group+1}")

    ax1 = plt.subplot(231)
    ax1.errorbar(x, ims_x - np.mean(ims_x), rms_x, fmt="blue", ecolor="black", lw=1.5)
    ax1.axhspan(-req_linear, req_linear, facecolor="gray", alpha=0.1)
    ax1.set_ylabel("x, \u03BCm", fontdict=font)
    ax1.set_xticks(x, xstring, rotation="vertical")

    ax2 = plt.subplot(232)
    ax2.errorbar(x, ims_y - np.mean(ims_y), rms_y, fmt="blue", ecolor="black", lw=1.5)
    ax2.axhspan(-req_linear, req_linear, facecolor="gray", alpha=0.1)
    ax2.set_ylabel("y, \u03BCm", fontdict=font)
    ax2.set_xticks(x, xstring, rotation="vertical")

    ax3 = plt.subplot(233)
    ax3.errorbar(x, ims_z - np.mean(ims_z), rms_z, fmt="blue", ecolor="black", lw=1.5)
    ax3.axhspan(-req_linear, req_linear, facecolor="gray", alpha=0.1)
    ax3.set_ylabel("z, \u03BCm", fontdict=font)
    ax3.set_xticks(x, xstring, rotation="vertical")

    ax4 = plt.subplot(234)
    ax4.errorbar(
        x, ims_xrot - np.mean(ims_xrot), rms_xrot, fmt="blue", ecolor="black", lw=1.5
    )
    ax4.axhspan(-req_angular, req_angular, facecolor="gray", alpha=0.1)
    ax4.set_ylabel("xRot, arcsec", fontdict=font)
    ax4.set_xticks(x, xstring, rotation="vertical")

    ax5 = plt.subplot(235)
    ax5.errorbar(
        x, ims_yrot - np.mean(ims_yrot), rms_yrot, fmt="blue", ecolor="black", lw=1.5
    )
    ax5.axhspan(-req_angular, req_angular, facecolor="gray", alpha=0.1)
    ax5.set_ylabel("yRot, arcsec", fontdict=font)
    ax5.set_xticks(x, xstring, rotation="vertical")

    ax6 = plt.subplot(236)
    ax6.errorbar(
        x, ims_zrot - np.mean(ims_zrot), rms_zrot, fmt="blue", ecolor="black", lw=1.5
    )
    ax6.axhspan(-req_angular, req_angular, facecolor="gray", alpha=0.1)
    ax6.set_ylabel("zRot, arcsec", fontdict=font)
    ax6.set_xticks(x, xstring, rotation="vertical")

    fig.tight_layout()

    ims = {
        'x': [(ims, rms) for ims, rms in zip(ims_x, rms_x)],
        'y': [(ims, rms) for ims, rms in zip(ims_y, rms_y)],
        'z': [(ims, rms) for ims, rms in zip(ims_z, rms_z)],
        'xRot': [(ims, rms) for ims, rms in zip(ims_xrot, rms_xrot)],
        'yRot': [(ims, rms) for ims, rms in zip(ims_yrot, rms_yrot)],
        'zRot': [(ims, rms) for ims, rms in zip(ims_zrot, rms_zrot)]
    }

    return ims

    

In [ ]:
# plot and fit tangent actuator encoder values

async def plot_encoder(
    efd_client,
    group,
    actuators_group,
    time_start,
    time_end,
    time_estop,
):
    topic = "lsst.sal.MTM2.axialEncoderPositions"
    topic_fields = await efd_client.get_fields(topic)

    query = efd_client.build_time_range_query(topic, topic_fields, time_start, time_end)
    query_df = await efd_client.influx_client.query(query)

    font = {
        "family": "serif",
        "color": "black",
        "weight": "normal",
        "size": 10,
    }

    fig, axs = plt.subplots(nrows=4, ncols=3)
    fig.suptitle(f"ACTUATORS GROUP {group+1}")
    c = 0
    j = 0
    for i in actuators_group:
        axs[j, c].plot(query_df.index, query_df[f"position{i}"])
        axs[j, c].plot(
            [time_estop, time_estop],
            [min(query_df[f"position{i}"]), max(query_df[f"position{i}"])],
            color="green",
            label="Estop",
        )
        axs[j, c].set_title("B=%i" % (i + 1), fontdict=font)
        axs[j, c].legend(loc="lower left", fontsize="5")
        axs[j, c].set_xlabel("Time", fontdict=font)
        axs[j, c].set_ylabel("Encoder, \u03BCm", fontdict=font)
        axs[j, c].get_xaxis().set_ticks([])

        c += 1
        if c > 2:
            j += 1
            c = 0

    fig.tight_layout()

    # linear fit encoder vs. time
    fig, axs = plt.subplots(nrows=4, ncols=3)
    fig.suptitle(f"ACTUATORS GROUP {group+1}")
    c = 0
    j = 0

    mlist = list()

    for i in actuators_group:
        encoder_drift = query_df[f"position{i}"][query_df.index > time_estop]
        x_post = np.linspace(0, len(encoder_drift), num=len(encoder_drift))
        y_post = encoder_drift.iloc[:].values
        coef = np.polyfit(x_post, y_post, 1)
        mlist.append((i, coef[0]))
        poly1d_fn = np.poly1d(coef)

        axs[j, c].plot(x_post, y_post, "yo", x_post, poly1d_fn(x_post), "--k")
        axs[j, c].set_title(f"B={i+1}, m={coef[0]:.2e}", fontdict=font)
        axs[j, c].set_xlabel("Time", fontdict=font)
        axs[j, c].set_ylabel("Encoder, \u03BCm", fontdict=font)
        axs[j, c].get_xaxis().set_ticks([])
        axs[j, c].get_yaxis().set_ticks([-2000, 2000])

        c += 1
        if c > 2:
            j += 1
            c = 0

    fig.tight_layout()

    return mlist

In [ ]:
async def plot_motorsteps(
    efd_client, time_start, time_end, group, actuators_group, time_estop
):
    # Plot motor steps

    topic = "lsst.sal.MTM2.axialActuatorSteps"
    topic_fields = await efd_client.get_fields(topic)

    query = efd_client.build_time_range_query(topic, topic_fields, time_start, time_end)
    query_df2 = await efd_client.influx_client.query(query)

    font = {
        "family": "serif",
        "color": "black",
        "weight": "normal",
        "size": 10,
    }

    fig, axs = plt.subplots(nrows=4, ncols=3)
    fig.suptitle(f"ACTUATORS GROUP {group+1}")
    c = 0
    j = 0
    for i in actuators_group:
        axs[j, c].plot(query_df2.index, query_df2[f"steps{i}"])
        axs[j, c].plot(
            [time_estop, time_estop],
            [min(query_df2[f"steps{i}"]), max(query_df2[f"steps{i}"])],
            color="green",
            label="Estop",
        )
        axs[j, c].set_title("B=%i" % (i + 1), fontdict=font)
        axs[j, c].legend(loc="lower left", fontsize="5")
        axs[j, c].set_ylabel("Steps", fontdict=font)
        axs[j, c].set_xlabel("Time", fontdict=font)
        axs[j, c].get_xaxis().set_ticks([])

        c += 1
        if c > 2:
            j += 1
            c = 0

        fig.tight_layout()

## Instantiate the MockControlClosedLoop

In [ ]:
config_path = Path.home() / "WORK" / "ts_mtm2" / "config" / "parameter_files"
filepath_cell_geom = Path.home() / "WORK" / "ts_mtm2" / "config" / "cellGeom.json"

In [ ]:
control_closed_loop = MockControlClosedLoop()
control_closed_loop.load_file_cell_geometry(filepath_cell_geom)

## Instantiate the DiagnosticsM2 Class

Notice that the UTC time is used when doing the query.

In [ ]:
diagnostics_m2 = DiagnosticsM2(efd_name=EfdName.Idf)

# Instatiate EFD client

In [ ]:
efd_client = lsst_efd_client.EfdClient(efd_name="idf_efd")

## Axial actuator groups

In [ ]:
actuators_groups = np.array(
    [
        [0, 10, 20, 4, 14, 24, 31, 39, 47, 56, 62, 68],
        [1, 11, 21, 5, 15, 25, 32, 40, 48, 57, 63, 69],
        [2, 12, 22, 6, 16, 26, 33, 38, 46, 54, 63, 69],
        [3, 13, 23, 7, 17, 27, 33, 41, 49, 58, 64, 70],
        [4, 14, 24, 8, 18, 28, 34, 42, 50, 59, 64, 70],
        [6, 16, 26, 9, 19, 29, 35, 43, 51, 60, 65, 71],
        [6, 16, 26, 10, 20, 30, 36, 44, 52, 61, 66, 0],
        [7, 17, 27, 11, 21, 31, 37, 45, 53, 55, 67, 0],
    ]
)

## M2 facing downward

In [ ]:
groups_downward = {
    0: [
        Time("2023-05-24T16:07:00", scale="utc", format="isot"),
        Time("2023-05-24T16:20:00", scale="utc", format="isot"),
    ],
    1: [
        Time("2023-05-24T16:45:00", scale="utc", format="isot"),
        Time("2023-05-24T16:55:00", scale="utc", format="isot"),
    ],
    2: [
        Time("2023-05-24T18:49:00", scale="utc", format="isot"),
        Time("2023-05-24T18:56:30", scale="utc", format="isot"),
    ],
    3: [
        Time("2023-05-24T19:06:20", scale="utc", format="isot"),
        Time("2023-05-24T19:14:00", scale="utc", format="isot"),
    ],
    4: [
        Time("2023-05-24T19:16:40", scale="utc", format="isot"),
        Time("2023-05-24T19:26:00", scale="utc", format="isot"),
    ],
    5: [
        Time("2023-05-24T19:27:40", scale="utc", format="isot"),
        Time("2023-05-24T19:35:45", scale="utc", format="isot"),
    ],
    6: [
        Time("2023-05-24T19:37:40", scale="utc", format="isot"),
        Time("2023-05-24T19:45:50", scale="utc", format="isot"),
    ],
    7: [
        Time("2023-05-24T20:08:15", scale="utc", format="isot"),
        Time("2023-05-24T20:18:55", scale="utc", format="isot"),
    ],
}

In [ ]:
m_encoder = {
    f'group{i}': {'downward': None, 'upward': None} for i in range(8)
}

In [ ]:
# run the analysis for axial actuators divided into groups for the M2 facing down
# the function also produces a table in rest format of the IMS RBP

print("*** \n MIRROR FACING DOWNWARD \n ***")
for key, val in groups_downward.items():
    print(f"- ACTUATORS GROUP N.{key+1} -")
    time_start = val[0]
    time_end = val[1]

    (
        xy_actuators,
        data_ims,
        data_collected_axial,
        data_power_status,
    ) = await query_data(diagnostics_m2, control_closed_loop, time_start, time_end)
    (
        full_stream,
        statistics_pre_estop,
        statistics_post_estop,
        eStop,
        eStop_ims,
        time_estop,
        force_command_time,
        command_time_ims,
    ) = print_data(
        data_collected_axial,
        data_ims,
        data_power_status,
        actuators_groups[key],
        key,
        "down",
    )

    actuator_plot(
        diagnostics_m2,
        control_closed_loop,
        force_command_time,
        actuators_groups[key],
        key,
        eStop,
        data_collected_axial,
        full_stream,
        statistics_pre_estop,
        statistics_post_estop,
        "down",
    )

    ims = ims_estop(
        data_ims,
        command_time_ims,
        eStop_ims,
        key,
    )

    m_encoder[f'group{key}']['downward'] = await plot_encoder(
            efd_client,
            key,
            actuators_groups[key],
            time_start,
            time_end,
            time_estop,
        )

    
    # write to file table in rest format
    
    with open("ims_table.txt", "a") as ims_tab_file:
        if key == 0:
            ims_tab_file.write("*** MIRROR FACING DOWNWARD ***\n")
        else:
            ims_tab_file.write("\n\n")

        ims_tab_file.write(f"***\nACTUATOR {key+1}\n")
        ims_tab_file.write(
            tabulate.tabulate(
                [(f'{ims[0][0]:.2f} +/- {ims[0][1]:.2f}', f'{ims[1][0]:.2} +/- {ims[1][1]:.2f}') for key, ims in ims.items() if key != "time"],
                headers=["DoF", "Command Time", "eStop"],
                showindex=list(ims.keys()),
                tablefmt="rst",
            )
        )

    await plot_motorsteps(
        efd_client, time_start, time_end, key, actuators_groups[key], time_estop
    )  

## M2 facing upward

In [ ]:
groups_upward = {
    0: [
        Time("2023-05-31T00:14:00", scale="utc", format="isot"),
        Time("2023-05-31T00:21:05", scale="utc", format="isot"),
    ],
    1: [
        Time("2023-05-31T00:38:10", scale="utc", format="isot"),
        Time("2023-05-31T00:47:00", scale="utc", format="isot"),
    ],
    2: [
        Time("2023-05-31T00:49:00", scale="utc", format="isot"),
        Time("2023-05-31T00:58:50", scale="utc", format="isot"),
    ],
    3: [
        Time("2023-05-31T01:01:25", scale="utc", format="isot"),
        Time("2023-05-31T01:10:50", scale="utc", format="isot"),
    ],
    4: [
        Time("2023-05-31T01:42:35", scale="utc", format="isot"),
        Time("2023-05-31T01:51:00", scale="utc", format="isot"),
    ],
    5: [
        Time("2023-05-31T01:52:55", scale="utc", format="isot"),
        Time("2023-05-31T02:01:45", scale="utc", format="isot"),
    ],
    6: [
        Time("2023-05-31T02:03:15", scale="utc", format="isot"),
        Time("2023-05-31T02:13:25", scale="utc", format="isot"),
    ],
    7: [
        Time("2023-05-31T02:16:20", scale="utc", format="isot"),
        Time("2023-05-31T02:27:00", scale="utc", format="isot"),
    ],
}

In [ ]:
# run the analysis for axial actuators divided into groups for the M2 facing up
# the function also produces a table in rest format of the IMS RBP

print("*** \n MIRROR FACING UPWARD \n ***")
for key, val in groups_upward.items():
    print(f"- ACTUATORS GROUP N.{key+1} -")
    time_start = val[0]
    time_end = val[1]

    (
        xy_actuators,
        data_ims,
        data_collected_axial,
        data_power_status,
    ) = await query_data(diagnostics_m2, control_closed_loop, time_start, time_end)
    (
        full_stream,
        statistics_pre_estop,
        statistics_post_estop,
        eStop,
        eStop_ims,
        time_estop,
        force_command_time,
        command_time_ims,
    ) = print_data(
        data_collected_axial,
        data_ims,
        data_power_status,
        actuators_groups[key],
        key,
        "up",
    )

    actuator_plot(
        diagnostics_m2,
        control_closed_loop,
        force_command_time,
        actuators_groups[key],
        key,
        eStop,
        data_collected_axial,
        full_stream,
        statistics_pre_estop,
        statistics_post_estop,
        "up",
    )

    ims = ims_estop(
        data_ims,
        command_time_ims,
        eStop_ims,
        key,
    )

    m_encoder[f'group{key}']['upward'] = await plot_encoder(
            efd_client,
            key,
            actuators_groups[key],
            time_start,
            time_end,
            time_estop,
        )

    
    # write to file table in rest format
    
    with open("ims_table.txt", "a") as ims_tab_file:
        if key == 0:
            ims_tab_file.write("\n\n*** MIRROR FACING UPWARD ***")
        else:
            ims_tab_file.write("\n\n")

        ims_tab_file.write(f"***\nACTUATOR {key+1}\n")
        ims_tab_file.write(
            tabulate.tabulate(
                [(f'{ims[0][0]:.2f} +/- {ims[0][1]:.2f}', f'{ims[1][0]:.2} +/- {ims[1][1]:.2f}') for key, ims in ims.items() if key != "time"],
                headers=["DoF", "Command Time", "eStop"],
                showindex=list(ims.keys()),
                tablefmt="rst",
            )
        )

    await plot_motorsteps(
        efd_client, time_start, time_end, key, actuators_groups[key], time_estop
    )

In [ ]:
# create a table of the angular coefficient from the encoder values linear 

with open("m_encoder_table.txt", "a") as menc_tab_file:
    for i, group in enumerate(m_encoder):
        menc_tab_file.write(f'*** ACTUATOR GROUP {i+1}***\n')
        menc_tab_file.write(
            tabulate.tabulate(
                [
                    (el_up[1], el_down[1])
                    for el_up, el_down in zip(*m_encoder[group].values())
                ],
                headers=["Id. Actuator", "Downdard", "Upward"],
                showindex=[el[0] for el in m_encoder[f"group{i}"]["downward"]],
                tablefmt="rst",
            )
        )
        menc_tab_file.write('\n')